# Perform structured extraction with LLM

First is to establish a baseline performance. Here we will use the Llama 3.1 70B Instruct model, a fairly capable LLM, to perform entity extraction on the evaluation dataset.

On Databricks, this model has `structured output` capability enabled. See doc for details [here](https://docs.databricks.com/en/machine-learning/model-serving/structured-outputs.html). 

In addition, we will use Precision, Recall, and F1 Score to evaluate the entity extraction result.

In [0]:
%pip install openai mlflow --quiet

dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import mlflow
mlflow.openai.autolog(log_traces=False)

In [0]:
input_table_name = "dhuang.synthgen.lease_docs_eval"
output_table_name = "dhuang.synthgen.lease_extracted_llama70b"
endpoint_name = "databricks-meta-llama-3-1-70b-instruct"
num_output_tokens = 4000
temperature = 0.0
input_column_name = "lease_doc"

## Prepare for entity extraction

In [0]:
eval_df = spark.sql(f"select * from {input_table_name}")
display(eval_df.limit(5))

lease_id lease_doc labels a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84 Exhibit 10.32
Lease Contract
(Translation)
Lessor: Shantou Yunjia Fashion Clothing Co., Ltd. (the “Party A”)
Lessee: Shantou Big Tree Toys Co., Ltd (the “Party B”)
In accordance with the relevant laws and regulations of the People’s Republic of China Contract Laws, with mutual agreement after negotiation, both parties has entered into this contract to define the rights and obligations that Party A and Party B shall respectively bear.
Section 1
Party A shall lease its property to Party B for business use which is located at South Part 1, Nanshe Area, Pengnan Industrial Park on North Yingbinbei Road in Waisha Town of Longhu District in Shantou City. The space area has 1,500 square meters.
Section 2 Lease Term
The term of this lease is 10 years. Party A shall release the property to Party B for use on January 1, 2011 and take back on December 31, 2021.
Party A shall have the rights to terminate the contract and take back the leased property if Party B has conducted one of the following activities:
1)
Sublease, underlease, transfer, lend, pool, contribute to an equity interest, or exchange the space with others without Party A’s permission.
2)
Use the rented property to conduct illegal activities and harm public interests.
After expiration of this lease contract, if Party A decides to continue leasing out the property, Party B has the right of first refusal to lease the property. When the contract expires and Party B fails to find another location to move to, Party B shall negotiate with Party A to extend the lease term.
Section 3 Rent and Payment Schedule
Party A and Party B agree that the monthly rent is RMB 4 per square meter. The annual rent is RMB 72,000 (US $ 11,340) and Party B shall make the annual payment by December 31 of each calendar year.
Section 4 Property Maintenance and Remodeling during the Term
Party A and Party B agree to adopt the third clause as follows to handle the matter of property maintenance:
1)
Party A shall be responsible for the maintenance at the expenses of Party A for the items agreed on by both parties.
2)
For the maintenance and remodeling projects allowed by Party A, Party B shall pay for the maintenance costs in advance and coordinate the construction. The advanced maintenance costs shall be deducted from the rent with the amount in the formal invoices.
3)
Party A shall be responsible for the maintenance project that cost more than RMB 5,000 and Party B shall be responsible for the regular maintenance of property during the lease term.
4)
Both parties negotiate to determine the responsibilities for property maintenance.
For its own use, Party B is allowed to remodel the rented property on the condition not to change the building structure. Party B can start construction only after obtaining the consent of Party A.
- 1 -
Section 5 Change of Both Parties
1)
When Party A follows legal procedures to transfer the ownership of the property to a third party, unless both parties have agreement otherwise, this contract continues to be binding on the new owners of the property.
2)
If Party A lists the property for sale, Party A shall give notice to Party B three months in advance. Under the same condition, Party B has the first right of refusal to purchase the property.
3)
If Party B wants to exchange the use of the property with a third party, Party B shall obtain the consent of Party A, and Party A shall support Party B’s reasonable requests.
Section 6 Breach of Contract
1)
If Party A fails to release the property to Party B in accordance with Section 1 and Section 2 of this contract, Party A shall pay 20% of the rent to Party B as penalty.
2)
If Party B delays rent payment, besides overdue rent Party B shall pay to Party A the penalty equivalent to 0.05% of the rent for each day after due date.
3)
Party B shall have the right to refuse to pay Party A any other expense except rent.
4)
If Party B subleases the rented space to any o

In [0]:
03example_lease = eval_df.select("lease_doc").limit(1).collect()[0]["lease_doc"]
example_labels = eval_df.select("labels").limit(1).collect()[0]["labels"]

print("Example Labels:")
print(example_labels)
print('-'*100)
print("Example Lease Agreement:")
print(example_lease)

Example Labels:
{
  "end_date": ["December 31, 2021"],
  "leased_space": ["property located at South Part 1, Nanshe Area, Pengnan Industrial Park on North Yingbinbei Road in Waisha Town of Longhu District in Shantou City. The space area has 1,500 square meters."],
  "lessee": ["Shantou Big Tree Toys Co., Ltd"],
  "lessor": ["Shantou Yunjia Fashion Clothing Co., Ltd"],
  "signing_date": [],
  "start_date": ["January 1, 2011"],
  "term_of_payment": ["Party A and Party B agree that the monthly rent is RMB 4 per square meter. The annual rent is RMB 72,000 (US $ 11,340) and Party B shall make the annual payment by December 31 of each calendar year."],
  "designated_use": ["business use"],
  "extension_period": ["After expiration of this lease contract, if Party A decides to continue leasing out the property, Party B has the right of first refusal to lease the property. When the contract expires and Party B fails to find another location to move to, Party B shall negotiate with Party A to ex

In [0]:
prompt = """You are an AI assistant specialized in analyzing legal contracts. 
Your task is to extract relevant information from a given contract document. 
Your output must be a structured JSON object.

Instructions:
1. Carefully read the entire contract document provided at the end of this prompt.
2. Extract the relevant information.
3. Present your findings in JSON format as specified below.

Important Notes:
- Extract only relevant information. 
- Consider the context of the entire contract when determining relevance.
- Do not be verbose, only respond with the correct format and information.
- Some docs may have multiple relevant excerpts -- include all that apply.
- Some questions may have no relevant excerpts -- just return ["N/A"].
- Do not include additional JSON keys beyond the ones listed here.
- Do not include the same key multiple times in the JSON.

Expected JSON keys and explanation of what they are:
- 'end_date': The end date of the lease.
- 'leased_space': Description of the space that is being leased.
- 'lessee': The lessee's name (and possibly address).
- 'lessor': The lessor's name (and possibly address).
- 'signing_date': The date the contract was signed.
- 'start_date': The start date of the lease.
- 'term_of_payment': Description of the payment terms.
- 'designated_use': Description of the designated use of the property being leased.
- 'extension_period': Description of the extension options for the lease.
- 'expiration_date_of_lease': The expiration data of the lease.

Contract to analyze: 
{lease}
"""

response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "lease_contract_extractions",
        "schema": {
            "type": "object",
            "properties": {
                "end_date": {
                    "type": "array", "items": {"type": "string"},
                    "description": "The end date of the lease.",
                },
                "leased_space": {
                    "type": "array", "items": {"type": "string"},
                    "description": "Description of the space that is being leased.",
                },
                "lessee": {
                    "type": "array", "items": {"type": "string"},
                    "description": "The lessee's name (and possibly address).",
                },
                "lessor": {
                    "type": "array", "items": {"type": "string"},
                    "description": "The lessor's name (and possibly address).",
                },
                "signing_date": {
                    "type": "array", "items": {"type": "string"},
                    "description": "The date the contract was signed.",
                },
                "start_date": {
                    "type": "array", "items": {"type": "string"},
                    "description": "The start date of the lease.",
                },
                "term_of_payment": {
                    "type": "array", "items": {"type": "string"},
                    "description": "Description of the payment terms.",
                },
                "designated_use": {
                    "type": "array", "items": {"type": "string"},
                    "description": "Description of the designated use of the property being leased.",
                },
                "extension_period": {
                    "type": "array", "items": {"type": "string"},
                    "description": "Description of the extension options for the lease.",
                },
                "expiration_date_of_lease": {
                    "type": "array", "items": {"type": "string"},
                    "description": "The expiration date of the lease.",
                },
            },
            "required": [
                "end_date", "leased_space", "lessee", "lessor", "signing_date", "start_date",
                "term_of_payment", "designated_use", "extension_period", "expiration_date_of_lease",
            ],
        },
        "strict": True,
    },
}

In [0]:
import openai
from mlflow.utils.databricks_utils import get_databricks_host_creds
creds = get_databricks_host_creds("databricks")

class Extractor:
    def __init__(self, creds, endpoint_name, prompt, temperature, num_output_tokens):
        self.client = openai.OpenAI(
            api_key=creds.token,
            base_url=f"{creds.host}/serving-endpoints",
            timeout=300,
            max_retries=3,
        )
        self.endpoint = endpoint_name
        self.prompt = prompt
        self.temperature = temperature
        self.num_output_tokens = num_output_tokens

    def predict(self, text):
        try:
            response = self.client.chat.completions.create(
                model=self.endpoint,
                messages=[{"role": "user", "content": self.prompt.format(lease=text)}],
                temperature=float(temperature),
                max_tokens=int(num_output_tokens),
                response_format=response_format,
            )        
            return (
                response.choices[0].message.content,
                response.usage.completion_tokens,
                response.usage.prompt_tokens,
                response.usage.total_tokens,
                None
            )

        except Exception as e:
            return None, 0, 0, 0, str(e)

In [0]:
outputs = Extractor(
    creds=creds,
    endpoint_name=endpoint_name,
    prompt=prompt,
    temperature=temperature,
    num_output_tokens=num_output_tokens
).predict(example_lease)

outputs

('{"designated_use": ["business use"], "end_date": ["December 31, 2021"], "expiration_date_of_lease": ["December 31, 2021"], "extension_period": ["After expiration of this lease contract, if Party A decides to continue leasing out the property, Party B has the right of first refusal to lease the property. When the contract expires and Party B fails to find another location to move to, Party B shall negotiate with Party A to extend the lease term."], "leased_space": ["South Part 1, Nanshe Area, Pengnan Industrial Park on North Yingbinbei Road in Waisha Town of Longhu District in Shantou City. The space area has 1,500 square meters."], "lessee": ["Shantou Big Tree Toys Co., Ltd"], "lessor": ["Shantou Yunjia Fashion Clothing Co., Ltd."], "signing_date": ["Not specified"], "start_date": ["January 1, 2011"], "term_of_payment": ["The monthly rent is RMB 4 per square meter. The annual rent is RMB 72,000 (US $ 11,340) and Party B shall make the annual payment by December 31 of each calendar ye

In [0]:
import json

json.loads(outputs[0])

{'designated_use': ['business use'],
 'end_date': ['December 31, 2021'],
 'expiration_date_of_lease': ['December 31, 2021'],
 'extension_period': ['After expiration of this lease contract, if Party A decides to continue leasing out the property, Party B has the right of first refusal to lease the property. When the contract expires and Party B fails to find another location to move to, Party B shall negotiate with Party A to extend the lease term.'],
 'leased_space': ['South Part 1, Nanshe Area, Pengnan Industrial Park on North Yingbinbei Road in Waisha Town of Longhu District in Shantou City. The space area has 1,500 square meters.'],
 'lessee': ['Shantou Big Tree Toys Co., Ltd'],
 'lessor': ['Shantou Yunjia Fashion Clothing Co., Ltd.'],
 'signing_date': ['Not specified'],
 'start_date': ['January 1, 2011'],
 'term_of_payment': ['The monthly rent is RMB 4 per square meter. The annual rent is RMB 72,000 (US $ 11,340) and Party B shall make the annual payment by December 31 of each cale

## Batch inference on your eval dataset

In [0]:
import time
import pandas as pd
from pyspark.sql.functions import pandas_udf
from typing import Iterator

endpoint_name = "databricks-meta-llama-3-1-70b-instruct"
temperature = 0.0
num_output_tokens = 4000

@pandas_udf(
    "output string, completion_tokens int, prompt_tokens int, total_tokens int, error string"
)
def extract_udf(content_batches: Iterator[pd.Series]) -> Iterator[pd.DataFrame]:
    client = Extractor(
        creds=creds,
        endpoint_name=endpoint_name,
        prompt=prompt,
        temperature=temperature,
        num_output_tokens=num_output_tokens,
    )
    for content_batch in content_batches:
        yield pd.DataFrame.from_records(content_batch.apply(client.predict))

In [0]:
import time
from pyspark.sql.functions import col

eval_df = eval_df.repartition(25)

batch_start_time = time.time()

extracted_df = (
    eval_df
    .withColumn("extraction", extract_udf(col("lease_doc")))
    .selectExpr(
        "*",
        "extraction.output as output",
        "extraction.completion_tokens as completion_tokens",
        "extraction.prompt_tokens as prompt_tokens",
        "extraction.total_tokens as total_tokens",
        "extraction.error as error",
    )
    .drop("extraction")
    .write.mode("overwrite")
    .saveAsTable(output_table_name)
)

batch_total_time = time.time() - batch_start_time

print(f"Batch time: {batch_total_time / 60 :.2f} mins")
print(f"Batch size: {eval_df.count()} files")
print(f"Batch rate: {eval_df.count() / (batch_total_time / 60):.2f} fpm")

Batch time: 2.41 mins
Batch size: 50 files
Batch rate: 20.72 fpm


In [0]:
%sql
-- look at output
select output from dhuang.synthgen.lease_extracted_llama70b version as of 3

output
"{""designated_use"": [""business use""], ""end_date"": [""December 31, 2021""], ""expiration_date_of_lease"": [""December 31, 2021""], ""extension_period"": [""After expiration of this lease contract, if Party A decides to continue leasing out the property, Party B has the right of first refusal to lease the property. When the contract expires and Party B fails to find another location to move to, Party B shall negotiate with Party A to extend the lease term.""], ""leased_space"": [""South Part 1, Nanshe Area, Pengnan Industrial Park on North Yingbinbei Road in Waisha Town of Longhu District in Shantou City. The space area has 1,500 square meters.""], ""lessee"": [""Shantou Big Tree Toys Co., Ltd""], ""lessor"": [""Shantou Yunjia Fashion Clothing Co., Ltd.""], ""signing_date"": [""N/A""], ""start_date"": [""January 1, 2011""], ""term_of_payment"": [""The monthly rent is RMB 4 per square meter. The annual rent is RMB 72,000 (US $ 11,340) and Party B shall make the annual payment by December 31 of each calendar year.""]}"
"{""designated_use"": [""workshop""], ""end_date"": [""July 31, 2023""], ""expiration_date_of_lease"": [""July 31, 2023""], ""extension_period"": [""Party B has the priority to lease the house after the contract expires, and it can extend the lease term with negotiation of Party A if it cannot find the house promptly after the contract expires.""], ""leased_space"": [""workshop (located at East Liuge Village, Weiwu Road, Bozhou, with the building area of 10000 square meters and usable area of 3000 square meters)""], ""lessee"": [""Anhui Xuelingxian Pharmaceutical Co., Ltd.""], ""lessor"": [""Bozhou Fengyi Chinese Medicine Development and Research Institute""], ""signing_date"": [""N/A""], ""start_date"": [""Aug 1, 2008""], ""term_of_payment"": [""The rent is 1.2 million Yuan, and Party B shall pay 100000 Yuan to Party A before 15 of each month as the monthly rent.""]}"
"{""designated_use"": [""general office, research, and development with commercial CLIA lab, warehouse, and distribution, and subject to the use restrictions identified in Sections 6 and 6.1, any lawful use related or incidental thereto""], ""end_date"": [""October 31st, 2018""], ""expiration_date_of_lease"": [""October 31st, 2018""], ""extension_period"": [""two (2), five (5) year options to extend the Term of this Lease""], ""leased_space"": [""28,994 rentable and 28,257 usable square feet""], ""lessee"": [""Exact Sciences Laboratories, Inc.""], ""lessor"": [""Tech Building I, LLC""], ""signing_date"": [""June 25, 2013""], ""start_date"": [""November 1st, 2013""], ""term_of_payment"": [""Base Rent set forth in Section 3.1 and Operating Charges set forth in Section 3.2 and any other additional payments due under this Lease""]}"
"{""designated_use"": [""Office""], ""end_date"": [""June 30, 2008""], ""expiration_date_of_lease"": [""June 30, 2008""], ""extension_period"": [""N/A""], ""leased_space"": [""2nd floor, Korea Design Center, 344-1, Yatap 1-dong, Bundang-gu, Seongnam, Kyunggi-do {space size-788.9 pyeong (2,608 square meter = 28,071 square feet)}""], ""lessee"": [""Leidis Technology Korea, Inc."", ""Suite 205 Korea Design Center, 344-1 Yatap 1-dong, Bundang-gu, Seongnam, Kyunggi-do"", ""Representative: Ahn, Sung Tae""], ""lessor"": [""Korea Institute of Design Promotion (KIDP)"", ""344-1, Yatap 1-dong, Bundang-gu, Seongnam, Kyunggi-do"", ""Representative: Lee, Il Kyu""], ""signing_date"": [""June 30, 2006""], ""start_date"": [""July 1, 2006""], ""term_of_payment"": [""Monthly Rent: KRW thirty one million and five hundred fifty six thousand (KRW 31,556,000 excluded VAT)"", ""Monthly Management Expenses: KRW fourteen million and nine hundred eighty nine thousand and one hundred (KRW 14,989,100 excluded VAT)""]}"
"{""designated_use"": [""The Tenant shall not use the Premises or any part of them except for the Permitted Use.""], ""end_date"": [""31 March 2031""], ""expiration_date_of_lease"": [""31 March 2031""], ""extension_period"": [

## Prepare outputs from Llama 70b for evaluation

A couple of considerations:
* Outputs are not always all clean -- some responses are needlessly verbose which produced invalid response
* Total tokens on this 68 samples are about 100k, which, dependso the LLM used, can be expensive if you apply this on a dataset that is 100x or 1000x larger

In [0]:
llama70b_df = spark.sql("select * from dhuang.synthgen.lease_extracted_llama70b version as of 3").select(
    "lease_id", "labels", "output"
)

display(llama70b_df)

lease_id,labels,output
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,"{ ""end_date"": [""December 31, 2021""], ""leased_space"": [""property located at South Part 1, Nanshe Area, Pengnan Industrial Park on North Yingbinbei Road in Waisha Town of Longhu District in Shantou City. The space area has 1,500 square meters.""], ""lessee"": [""Shantou Big Tree Toys Co., Ltd""], ""lessor"": [""Shantou Yunjia Fashion Clothing Co., Ltd""], ""signing_date"": [], ""start_date"": [""January 1, 2011""], ""term_of_payment"": [""Party A and Party B agree that the monthly rent is RMB 4 per square meter. The annual rent is RMB 72,000 (US $ 11,340) and Party B shall make the annual payment by December 31 of each calendar year.""], ""designated_use"": [""business use""], ""extension_period"": [""After expiration of this lease contract, if Party A decides to continue leasing out the property, Party B has the right of first refusal to lease the property. When the contract expires and Party B fails to find another location to move to, Party B shall negotiate with Party A to extend the lease term.""], ""expiration_date_of_lease"": [""December 31, 2021""] }","{""designated_use"": [""business use""], ""end_date"": [""December 31, 2021""], ""expiration_date_of_lease"": [""December 31, 2021""], ""extension_period"": [""After expiration of this lease contract, if Party A decides to continue leasing out the property, Party B has the right of first refusal to lease the property. When the contract expires and Party B fails to find another location to move to, Party B shall negotiate with Party A to extend the lease term.""], ""leased_space"": [""South Part 1, Nanshe Area, Pengnan Industrial Park on North Yingbinbei Road in Waisha Town of Longhu District in Shantou City. The space area has 1,500 square meters.""], ""lessee"": [""Shantou Big Tree Toys Co., Ltd""], ""lessor"": [""Shantou Yunjia Fashion Clothing Co., Ltd.""], ""signing_date"": [""N/A""], ""start_date"": [""January 1, 2011""], ""term_of_payment"": [""The monthly rent is RMB 4 per square meter. The annual rent is RMB 72,000 (US $ 11,340) and Party B shall make the annual payment by December 31 of each calendar year.""]}"
a0Xts4kMzdTGCoSkzyR4Ag3s93q0-lease_contract_201,"{ ""end_date"": [""July 31, 2023""], ""leased_space"": [""the workshop (located at East Liuge Village, Weiwu Road, Bozhou, with the building area of 10000 square meters and usable area of 3000 square meters)""], ""lessee"": [""Anhui Xuelingxian Pharmaceutical Co., Ltd.""], ""lessor"": [""Bozhou Fengyi Chinese Medicine Development and Research Institute""], ""signing_date"": [], ""start_date"": [""Aug 1, 2008""], ""term_of_payment"": [""The rent is 1.2 million Yuan, and Party B shall pay 100000 Yuan to Party A before 15 of each month as the monthly rent.""], ""designated_use"": [], ""extension_period"": [""Party B has the priority to lease the house after the contract expires, and it can extend the lease term with negotiation of Party A if it cannot find the house promptly after the contract expires.""], ""expiration_date_of_lease"": [""July 31, 2023""] }","{""designated_use"": [""workshop""], ""end_date"": [""July 31, 2023""], ""expiration_date_of_lease"": [""July 31, 2023""], ""extension_period"": [""Party B has the priority to lease the house after the contract expires, and it can extend the lease term with negotiation of Party A if it cannot find the house promptly after the contract expires.""], ""leased_space"": [""workshop (located at East Liuge Village, Weiwu Road, Bozhou, with the building area of 10000 square meters and usable area of 3000 square meters)""], ""lessee"": [""Anhui Xuelingxian Pharmaceutical Co., Ltd.""], ""lessor"": [""Bozhou Fengyi Chinese Medicine Development and Research Institute""], ""signing_date"": [""N/A""], ""start_date"": [""Aug 1, 2008""], ""term_of_payment"": [""The rent is 1.2 million Yuan, and Party B shall pay 100000 Yuan to Party A before 15 of each month as the monthly rent.""]}"
a3HA2M9pF0nn

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

fields = [
    "end_date",
    "leased_space",
    "lessee",
    "lessor",
    "signing_date",
    "start_date",
    "term_of_payment",
    "designated_use",
    "extension_period",
    "expiration_date_of_lease",
]

schema = StructType([StructField(field, ArrayType(StringType()), True) for field in fields])

In [0]:
import pandas as pd
from pyspark.sql.functions import from_json, col

def create_melted_df(df, column_name, schema):
    melted_df = (
        df.select("lease_id", column_name)
        .withColumn(column_name, from_json(col(column_name), schema))
        .select("lease_id", f"{column_name}.*")
        .toPandas()
    )
    return pd.melt(melted_df, id_vars=["lease_id"], var_name="variable", value_name=column_name)

# create labels df
labels_melted_df = create_melted_df(llama70b_df, "labels", schema)

# create outputs df
output_melted_df = create_melted_df(llama70b_df, "output", schema)

In [0]:
# combine both dfs
comp_df = (
    pd.merge(
        labels_melted_df,
        output_melted_df,
        left_on=["lease_id", "variable"],
        right_on=["lease_id", "variable"],
        how="left"
    )
    .sort_values(by="lease_id")
)

# clean up nulls and unpack arrays
comp_df['labels'] = comp_df['labels'].apply(lambda x: ', '.join(x) if x is not None else 'N/A')
comp_df['output'] = comp_df['output'].apply(lambda x: ', '.join(x) if x is not None else 'N/A')

display(comp_df.head(10))

lease_id,variable,labels,output
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,end_date,"December 31, 2021","December 31, 2021"
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,designated_use,business use,business use
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,term_of_payment,"Party A and Party B agree that the monthly rent is RMB 4 per square meter. The annual rent is RMB 72,000 (US $ 11,340) and Party B shall make the annual payment by December 31 of each calendar year.","The monthly rent is RMB 4 per square meter. The annual rent is RMB 72,000 (US $ 11,340) and Party B shall make the annual payment by December 31 of each calendar year."
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,start_date,"January 1, 2011","January 1, 2011"
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,signing_date,,N/A
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,expiration_date_of_lease,"December 31, 2021","December 31, 2021"
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,lessor,"Shantou Yunjia Fashion Clothing Co., Ltd","Shantou Yunjia Fashion Clothing Co., Ltd."
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,lessee,"Shantou Big Tree Toys Co., Ltd","Shantou Big Tree Toys Co., Ltd"
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,leased_space,"property located at South Part 1, Nanshe Area, Pengnan Industrial Park on North Yingbinbei Road in Waisha Town of Longhu District in Shantou City. The space area has 1,500 square meters.","South Part 1, Nanshe Area, Pengnan Industrial Park on North Yingbinbei Road in Waisha Town of Longhu District in Shantou City. The space area has 1,500 square meters."
a.7gwt0jast7_yYkZGNMh7zbFZZ8-lease_contract_84,extension_period,"After expiration of this lease contract, if Party A decides to continue leasing out the property, Party B has the right of first refusal to lease the property. When the contract expires and Party B fails to find another location to move to, Party B shall negotiate with Party A to extend the lease term.","After expiration of this lease contract, if Party A decides to continue leasing out the property, Party B has the right of first refusal to lease the property. When the contract expires and Party B fails to find another location to move to, Party B shall negotiate with Party A to extend the lease term."


## Evaluate accuracy

In [0]:
import difflib

def is_null_like(value):
    """Helper function to determine if a value is null-like."""
    return value is None or str(value).strip().lower() in [
        "n/a", "none", "null", "nan", "",
    ]

def compute_value_metrics(y_true, y_pred):
    tp = fn = fp = tn = 0

    for expected_value, generated_value in zip(y_true, y_pred):
        # Case-1: True Positive
        expected_null = is_null_like(expected_value)
        generated_null = is_null_like(generated_value)

        if not expected_null and not generated_null:
            # Fuzzy match instead of exact match
            similarity = difflib.SequenceMatcher(
                None, str(expected_value), str(generated_value)
            ).ratio()
            if similarity > 0.8:
                tp += 1
            else:
                fp += 1

        # Case-2: False Positive
        if not generated_null and expected_null:
            fp += 1

        # Case-3: False Negative
        if generated_null and not expected_null:
            fn += 1

        # Case-4: True Negative
        if generated_null and expected_null:
            tn += 1

    # Calculate additional metrics
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = (
        2 * (precision * recall) / (precision + recall)
        if (precision + recall) > 0
        else 0
    )

    return {
        "true_positive": tp,
        "fale_negative": fn,
        "false_positive": fp,
        "true_negative": tn,
        "presicion": round(precision, 4),
        "recall": round(recall, 4),
        "f1_score": round(f1_score, 4),
    }

In [0]:
y_true = comp_df["labels"].tolist()
y_pred = comp_df["output"].tolist()

compute_value_metrics(y_true, y_pred)

{'true_positive': 267,
 'fale_negative': 21,
 'false_positive': 202,
 'true_negative': 10,
 'presicion': 0.5693,
 'recall': 0.9271,
 'f1_score': 0.7054}

## Recap
Here we see that the baseline F1 Score is 61%. In the next notebook we will use the training dataset to create synthetic samples, and then use all available non-eval dataset to fine-tune a LLM on this task.